In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd

#Pre-processing data

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/qwen_3b_data_with_answer.csv')
df.head()

,question,answer,ground_truths
0,What are the typical winds in Genoa ?,<|im_start|>system\nYou are a knowledgeable as...,The typical winds in Genoa are in winter from ...
1,What is the depth of the sea in the port of Al...,<|im_start|>system\nYou are a knowledgeable as...,The average depth in the port of Alassio is 4m
2,What is the VHF channel of the port of imperia ?,<|im_start|>system\nYou are a knowledgeable as...,There are two VHF channels in the port of Impe...
3,Where is situated the Golfo Marconi ?,<|im_start|>system\nYou are a knowledgeable as...,Marconi Gulf is located between Portofino and ...
4,Can you tell me the email address of the port ...,<|im_start|>system\nYou are a knowledgeable as...,Ischia Harbor Email is ischia@guardiacostiera.it


In [ ]:
df['answer'][0]

"<|im_start|>system\nYou are a knowledgeable assistant specializing in answering questions about ports, sailing, and maritime activities along the Italian peninsula. Use the most relevant pieces of the retrieved context below to support your answer. If you don't know the answer or the context is insufficient, state that you are unsure. Use a maximum of three sentences, and keep your answer concise, accurate, and where applicable, use maritime terminology.<|im_end|>\n<|im_start|>user\nQuestion: What are the typical winds in Genoa ? \nContext: French border) to Genova, then SE and S to Promontorio diPiombino and Arcipelago Toscano. \nGeneral Remarks\n10.1 Winds—Weather.— In the Ligurian Sea and Golfe di\nGenova, the prevailing wind is from the NW practically all\nyear. However, locally along the coast, there is some variation\ndue to the land configuration. In the N part of the Tyrrhenian\n\nPunta della Madonnetta, 1 mile WNW, and consists of two har-bors, Porto del Valle, and Porto Vecc

In [ ]:
DEFAULT_RAG_PROMPT = """\
You are an assistant for question-answering tasks about maritime domain. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\
"""

QWEN_DEFAULT_RAG_PROMPT = """\
You are a knowledgeable assistant specializing in answering questions about ports, sailing, and maritime activities along the Italian peninsula. Use the most relevant pieces of the retrieved context below to support your answer. If you don't know the answer or the context is insufficient, state that you are unsure. Use a maximum of three sentences, and keep your answer concise, accurate, and where applicable, use maritime terminology."""

phi_rag_prompt_template = f"""<|system|>
{DEFAULT_RAG_PROMPT }<|end|>""" + """
<|user|>
Question: {input}
Context: {context} <|end|>
<|assistant|>"""

qwen_rag_prompt_template = f"""<|im_start|>system
{QWEN_DEFAULT_RAG_PROMPT }<|im_end|>""" + """
<|im_start|>user
Question: {input}
Context: {context}
<|im_end|>
<|im_start|>assistant<|im_end|>
"""

gemma_rag_prompt_template = f"""
{DEFAULT_RAG_PROMPT},""" + """<start_of_turn>user
{input}<end_of_turn><start_of_turn>context
{context}<end_of_turn>
<start_of_turn>model"""


model_dict = {
    "gemma": {
        "model_path": "../model/google/gemma-2-2b-it",
        "prompt_template": gemma_rag_prompt_template,
        "default_rag_prompt": DEFAULT_RAG_PROMPT
    },
    "phi_int4": {
        "model_path": "../model/microsoft/Phi-3.5-mini-instruct/int4",
        "prompt_template": phi_rag_prompt_template,
        "default_rag_prompt": DEFAULT_RAG_PROMPT
    },
    "phi_int8": {
        "model_path": "../model/microsoft/Phi-3.5-mini-instruct/int8",
        "prompt_template": phi_rag_prompt_template,
        "default_rag_prompt": DEFAULT_RAG_PROMPT
    },
    "qwen_0.5b": {
        "model_path": "../model/Qwen/Qwen2.5-0.5B-Instruct",
        "prompt_template": qwen_rag_prompt_template,
        "default_rag_prompt": QWEN_DEFAULT_RAG_PROMPT
    },
    "qwen_1.5b": {
        "model_path": "../model/Qwen/Qwen2.5-1.5B-Instruct",
        "prompt_template": qwen_rag_prompt_template,
        "default_rag_prompt": QWEN_DEFAULT_RAG_PROMPT
    },
    "qwen_3b": {
        "model_path": "../model/Qwen/Qwen2.5-3B-Instruct/int8",
        "prompt_template": qwen_rag_prompt_template,
        "default_rag_prompt": QWEN_DEFAULT_RAG_PROMPT
    }
}

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import logging
from datasets import Dataset

def proc_datacreated_by_qwen():
    filtered_dict = {k: v for k, v in model_dict.items() if k.startswith('qwen')}
    for model_name, _ in filtered_dict.items():
        # TODO: load the data from each file and create a single dataset (maybe not in csv format)

        df = pd.read_csv(f"/gdrive/MyDrive/Tesi_magistrale/{model_name}_data_with_answer.csv")

        # Initialize lists to store the cleaned answers and contexts
        cleaned_answers = []
        contexts = []

        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            answer_text = row['answer']

            # Split the answer text based on the markers <|im_start|> and <|im_end|>
            if "<|im_start|>user" in answer_text and "<|im_start|>assistant<|im_end|>" in answer_text:
                # Extract context and the cleaned answer
                context_part = answer_text.split("Context:")[1].split("<|im_end|>")[0].strip()
                answer_part = answer_text.split("<|im_start|>assistant<|im_end|>")[1].strip()

                # Append to the lists
                cleaned_answers.append(answer_part)
                contexts.append([context_part])
            else:
                # If markers are not found, append the original content
                cleaned_answers.append(answer_text)
                contexts.append([""])

        print(len(contexts))
        # Create a new dictionary with cleaned answers and contexts
        dictionary = {
            "question": df['question'].tolist(),
            "ground_truth": df['ground_truths'].tolist(),
            "answer": cleaned_answers,
            "contexts": contexts

        }
        try:
            #create csv file
            save_data = pd.DataFrame.from_dict(dictionary)
            save_path = f"{model_name}_data_cleaned.csv"
            save_data.to_csv(save_path, index=False)
        except Exception as e:
            logging.error("Error saving the data: {}".format(e))

    logging.info("Data cleaning completed successfully.")


proc_datacreated_by_qwen()

25
25
25


In [ ]:
df_cleaned = pd.read_csv('/content/qwen_3b_data_cleaned.csv')
df_cleaned.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,<tool_call>user\nBased on the provided context...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,<tool_call>user\nThe depth of the sea in the p...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,"According to the provided information, the VHF...",['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,user\nWhere is situated the Golfo Marconi?\nTh...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"Based on the provided contact information, the...",['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_cleaned['answer']

,answer
0,<tool_call>user\nBased on the provided context...
1,<tool_call>user\nThe depth of the sea in the p...
2,"According to the provided information, the VHF..."
3,user\nWhere is situated the Golfo Marconi?\nTh...
4,"Based on the provided contact information, the..."
5,"To enter the port of Genoa, vessels should app..."
6,"<tool_call>user\nI apologize, but the provided..."
7,user\nThe main dangers in the port of Naples i...
8,"user\nBased on the provided context, the harbo..."
9,<tool_call>user\nThe island of Pianosa is part...


In [ ]:
# Iterate over each row in the DataFrame to process the answers
for index, row in df_cleaned.iterrows():
    answer_text = row['answer']

    # Check if the answer contains the patterns and trim appropriately
    if "<tool_call>user\n" in answer_text:
        answer_part = answer_text.split("<tool_call>user\n", 1)[1].strip()
    elif "user\n" in answer_text:
        answer_part = answer_text.split("user\n", 1)[1].strip()
    else:
        answer_part = answer_text.strip()

    # Update the answer in the DataFrame
    df_cleaned.at[index, 'answer'] = answer_part

# Display the modified DataFrame
df_cleaned.head()


,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,"Based on the provided context, the prevailing ...","['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,"According to the provided information, the VHF...",['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,Where is situated the Golfo Marconi?\nThe Golf...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"Based on the provided contact information, the...",['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_cleaned['answer']

,answer
0,"Based on the provided context, the prevailing ..."
1,The depth of the sea in the port of Alassio is...
2,"According to the provided information, the VHF..."
3,Where is situated the Golfo Marconi?\nThe Golf...
4,"Based on the provided contact information, the..."
5,"To enter the port of Genoa, vessels should app..."
6,"I apologize, but the provided context does not..."
7,The main dangers in the port of Naples include...
8,"Based on the provided context, the harbor of O..."
9,The island of Pianosa is part of the Archipela...


In [ ]:
df_1_5 = pd.read_csv('/content/qwen_1.5b_data_cleaned.csv')
df_1_5.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,user\nIs it true that Genoa experiences strong...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,user\nWhat is the depth of the sea in the port...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,user\nWhat is the VHF channel for the port of ...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,user\nCould you please provide more informatio...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"user\nThank you for the information, but I'm a...",['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
for index, row in df_1_5.iterrows():
    answer_text = row['answer']

    # Check if the answer contains the patterns and trim appropriately

    if "user\n" in answer_text:
        answer_part = answer_text.split("user\n", 1)[1].strip()
    else:
        answer_part = answer_text.strip()

    # Update the answer in the DataFrame
    df_1_5.at[index, 'answer'] = answer_part

# Display the modified DataFrame
df_1_5.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,Is it true that Genoa experiences strong winds...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,What is the depth of the sea in the port of Al...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,What is the VHF channel for the port of Imperi...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,Could you please provide more information on t...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"Thank you for the information, but I'm actuall...",['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_1_5['answer']

,answer
0,Is it true that Genoa experiences strong winds...
1,What is the depth of the sea in the port of Al...
2,What is the VHF channel for the port of Imperi...
3,Could you please provide more information on t...
4,"Thank you for the information, but I'm actuall..."
5,Can you provide guidance on entering the port ...
6,What is the intensity of the tides in the port...
7,Can you provide some information on the histor...
8,What wind conditions can be expected at Otrant...
9,Is there any chance to visit the island of Pia...


In [ ]:
df_0_5 = pd.read_csv('/content/qwen_0.5b_data_cleaned.csv')
df_0_5.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,"Based on the provided context, the typical win...","['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channel for the port of Imperia is 12....,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,NaN,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the port of Ischia is pr...,['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_0_5['answer']

,answer
0,"Based on the provided context, the typical win..."
1,The depth of the sea in the port of Alassio is...
2,The VHF channel for the port of Imperia is 12....
3,NaN
4,The email address for the port of Ischia is pr...
5,"To enter the port of Genoa, you need to follow..."
6,To determine the intensity of the tides in the...
7,The main dangers associated with the port of N...
8,"Based on the provided context, the winds that ..."
9,"Yes, it is possible to land on the island of P..."


In [ ]:
df_cleaned.to_csv('qwen_3b_data_cleaned.csv', index=False)
df_1_5.to_csv('qwen_1.5b_data_cleaned.csv', index=False)


In [ ]:
df_cleaned = pd.read_csv('/content/qwen_3b_data_cleaned.csv')
df_cleaned.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,"Based on the provided context, the prevailing ...","['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,"According to the provided information, the VHF...",['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,Where is situated the Golfo Marconi?\nThe Golf...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"Based on the provided contact information, the...",['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_1_5 = pd.read_csv('/content/qwen_1.5b_data_cleaned.csv')
df_1_5.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,Is it true that Genoa experiences strong winds...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,What is the depth of the sea in the port of Al...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,What is the VHF channel for the port of Imperi...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,Could you please provide more information on t...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"Thank you for the information, but I'm actuall...",['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_1_5['answer']

,answer
0,Is it true that Genoa experiences strong winds...
1,What is the depth of the sea in the port of Al...
2,What is the VHF channel for the port of Imperi...
3,Could you please provide more information on t...
4,"Thank you for the information, but I'm actuall..."
5,Can you provide guidance on entering the port ...
6,What is the intensity of the tides in the port...
7,Can you provide some information on the histor...
8,What wind conditions can be expected at Otrant...
9,Is there any chance to visit the island of Pia...


In [ ]:
df_phi_int4 = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/phi_int4_data_with_answer.csv')
df_phi_int4.head()

,question,answer,ground_truths
0,What are the typical winds in Genoa ?,<|system|> \nYou are an assistant for question...,The typical winds in Genoa are in winter from ...
1,What is the depth of the sea in the port of Al...,<|system|> \nYou are an assistant for question...,The average depth in the port of Alassio is 4m
2,What is the VHF channel of the port of imperia ?,<|system|> \nYou are an assistant for question...,There are two VHF channels in the port of Impe...
3,Where is situated the Golfo Marconi ?,<|system|> \nYou are an assistant for question...,Marconi Gulf is located between Portofino and ...
4,Can you tell me the email address of the port ...,<|system|> \nYou are an assistant for question...,Ischia Harbor Email is ischia@guardiacostiera.it


In [ ]:
df_phi_int4['answer']

,answer
0,<|system|> \nYou are an assistant for question...
1,<|system|> \nYou are an assistant for question...
2,<|system|> \nYou are an assistant for question...
3,<|system|> \nYou are an assistant for question...
4,<|system|> \nYou are an assistant for question...
5,<|system|> \nYou are an assistant for question...
6,<|system|> \nYou are an assistant for question...
7,<|system|> \nYou are an assistant for question...
8,<|system|> \nYou are an assistant for question...
9,<|system|> \nYou are an assistant for question...


In [ ]:
df_phi_int4['answer'][0]

"<|system|> \nYou are an assistant for question-answering tasks about maritime domain. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.<|end|>\n<|user|>\nQuestion: What are the typical winds in Genoa ? \nContext: French border) to Genova, then SE and S to Promontorio diPiombino and Arcipelago Toscano. \nGeneral Remarks\n10.1 Winds—Weather.— In the Ligurian Sea and Golfe di\nGenova, the prevailing wind is from the NW practically all\nyear. However, locally along the coast, there is some variation\ndue to the land configuration. In the N part of the Tyrrhenian\n\nPunta della Madonnetta, 1 mile WNW, and consists of two har-bors, Porto del Valle, and Porto Vecchio.\n13.9Winds—Weather.— Winds from the NW are often strong\nand raise a sea in the roadst ead. Winds from the NE quadrant,\nespecially NNE winds, are dangerous to vessels at the quays.\n13.9Depth

In [ ]:
def proc_data_created_by_phi():
    filtered_dict = {k: v for k, v in model_dict.items() if k.startswith('phi')}

    for model_name, _ in filtered_dict.items():
        # TODO: load the data from each file and create a single dataset (maybe not in csv format)

        df = pd.read_csv(f"/gdrive/MyDrive/Tesi_magistrale/{model_name}_data_with_answer.csv")

        # Initialize lists to store the cleaned answers and contexts
        cleaned_answers = []
        contexts = []

        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            answer_text = row['answer']

            # Split the answer text based on the markers <|im_start|> and <|im_end|>
            if "<|user|>" in answer_text and "<|assistant|>" in answer_text:
                # Extract context and the cleaned answer
                context_part = answer_text.split("Context:")[1].split("<|end|>")[0].strip()
                answer_part = answer_text.split("<|assistant|>")[1].strip()

                # Append to the lists
                cleaned_answers.append(answer_part)
                contexts.append([context_part])
            else:
                # If markers are not found, append the original content
                cleaned_answers.append(answer_text)
                contexts.append([""])

        print(len(contexts))
        # Create a new dictionary with cleaned answers and contexts
        dictionary = {
            "question": df['question'].tolist(),
            "ground_truth": df['ground_truths'].tolist(),
            "answer": cleaned_answers,
            "contexts": contexts

        }
        try:
            #create csv file
            save_data = pd.DataFrame.from_dict(dictionary)
            save_path = f"{model_name}_data_cleaned.csv"
            save_data.to_csv(save_path, index=False)
        except Exception as e:
            logging.error("Error saving the data: {}".format(e))

    logging.info("Data cleaning completed successfully.")

proc_data_created_by_phi()

25
25


In [ ]:
df_phi_int4_cleaned = pd.read_csv('/content/phi_int4_data_cleaned.csv')
df_phi_int4_cleaned.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,The typical winds in Genoa are predominantly f...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio ra...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channels of the port of Imperia are ch...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,The Golfo Marconi is situated between Punta di...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the port of Ischia is uc...,['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_phi_int4_cleaned['answer']

,answer
0,The typical winds in Genoa are predominantly f...
1,The depth of the sea in the port of Alassio ra...
2,The VHF channels of the port of Imperia are ch...
3,The Golfo Marconi is situated between Punta di...
4,The email address for the port of Ischia is uc...
5,"To enter the port of Genoa, large vessels shou..."
6,The provided context does not explicitly menti...
7,The main dangers in the port of Naples include...
8,The harbor of Otranto is sheltered from N and ...
9,"Yes, it is possible to land on the island of P..."


In [ ]:
df_phi_int4_cleaned['answer'][0]

'The typical winds in Genoa are predominantly from the northwest, often strong and potentially dangerous, especially for vessels near the quays. However, there can be local variations due to the coastal land configuration. The prevailing winds can raise a sea in the roadside area.'

In [ ]:
df_phi_int8_cleaned = pd.read_csv('/content/phi_int8_data_cleaned.csv')
df_phi_int8_cleaned.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,The typical winds in Genoa are predominantly f...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channels of the port of Imperia are ch...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,"The Golfo Marconi is situated at 44°18'N., 9°1...","[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the port of Ischia is uc...,['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_phi_int8_cleaned['answer'][0]

'The typical winds in Genoa are predominantly from the northwest (NW) throughout the year. However, there can be variations along the coast due to land configuration. Strong NW winds can raise waves, posing challenges for vessels near the quays.'

In [ ]:
df_phi_int8_cleaned['contexts'][0]

"['French border) to Genova, then SE and S to Promontorio diPiombino and Arcipelago Toscano. \\nGeneral Remarks\\n10.1 Winds—Weather.— In the Ligurian Sea and Golfe di\\nGenova, the prevailing wind is from the NW practically all\\nyear. However, locally along the coast, there is some variation\\ndue to the land configuration. In the N part of the Tyrrhenian\\n\\nPunta della Madonnetta, 1 mile WNW, and consists of two har-bors, Porto del Valle, and Porto Vecchio.\\n13.9Winds—Weather.— Winds from the NW are often strong\\nand raise a sea in the roadst ead. Winds from the NE quadrant,\\nespecially NNE winds, are dangerous to vessels at the quays.\\n13.9Depths—Limitations.— The entrance to Porto del Valle has']"

In [ ]:
df_phi_int4_cleaned['contexts'][0]

"['French border) to Genova, then SE and S to Promontorio diPiombino and Arcipelago Toscano. \\nGeneral Remarks\\n10.1 Winds—Weather.— In the Ligurian Sea and Golfe di\\nGenova, the prevailing wind is from the NW practically all\\nyear. However, locally along the coast, there is some variation\\ndue to the land configuration. In the N part of the Tyrrhenian\\n\\nPunta della Madonnetta, 1 mile WNW, and consists of two har-bors, Porto del Valle, and Porto Vecchio.\\n13.9Winds—Weather.— Winds from the NW are often strong\\nand raise a sea in the roadst ead. Winds from the NE quadrant,\\nespecially NNE winds, are dangerous to vessels at the quays.\\n13.9Depths—Limitations.— The entrance to Porto del Valle has']"

In [ ]:
df_phi_int4_cleaned.to_csv('phi_int4_data_cleaned.csv', index=False)
df_phi_int8_cleaned.to_csv('phi_int8_data_cleaned.csv', index=False)

In [ ]:
df_phi_int4_cleaned = pd.read_csv('/content/phi_int4_data_cleaned.csv')
df_phi_int4_cleaned.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,The typical winds in Genoa are predominantly f...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio ra...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channels of the port of Imperia are ch...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,The Golfo Marconi is situated between Punta di...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the port of Ischia is uc...,['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_phi_int8_cleaned = pd.read_csv('/content/phi_int8_data_cleaned.csv')
df_phi_int8_cleaned.head()


,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,The typical winds in Genoa are predominantly f...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channels of the port of Imperia are ch...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,"The Golfo Marconi is situated at 44°18'N., 9°1...","[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the port of Ischia is uc...,['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_gemma = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/gemma_data_with_answer.csv')
df_gemma.head()

,question,answer,ground_truths
0,What are the typical winds in Genoa ?,\nYou are an assistant for question-answering ...,The typical winds in Genoa are in winter from ...
1,What is the depth of the sea in the port of Al...,\nYou are an assistant for question-answering ...,The average depth in the port of Alassio is 4m
2,What is the VHF channel of the port of imperia ?,\nYou are an assistant for question-answering ...,There are two VHF channels in the port of Impe...
3,Where is situated the Golfo Marconi ?,\nYou are an assistant for question-answering ...,Marconi Gulf is located between Portofino and ...
4,Can you tell me the email address of the port ...,\nYou are an assistant for question-answering ...,Ischia Harbor Email is ischia@guardiacostiera.it


In [ ]:
df_gemma['answer'][0]

"\nYou are an assistant for question-answering tasks about maritime domain. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.,<start_of_turn>user\nWhat are the typical winds in Genoa ?<end_of_turn><start_of_turn>context\nFrench border) to Genova, then SE and S to Promontorio diPiombino and Arcipelago Toscano. \nGeneral Remarks\n10.1 Winds—Weather.— In the Ligurian Sea and Golfe di\nGenova, the prevailing wind is from the NW practically all\nyear. However, locally along the coast, there is some variation\ndue to the land configuration. In the N part of the Tyrrhenian\n\nPunta della Madonnetta, 1 mile WNW, and consists of two har-bors, Porto del Valle, and Porto Vecchio.\n13.9Winds—Weather.— Winds from the NW are often strong\nand raise a sea in the roadst ead. Winds from the NE quadrant,\nespecially NNE winds, are dangerous to vessels at the quays.\n13

In [ ]:
def proc_data_created_by_gemma():
    filtered_dict = {k: v for k, v in model_dict.items() if k.startswith('gemma')}

    for model_name, _ in filtered_dict.items():
        # TODO: load the data from each file and create a single dataset (maybe not in csv format)

        df = pd.read_csv(f"/gdrive/MyDrive/Tesi_magistrale/{model_name}_data_with_answer.csv")

        # Initialize lists to store the cleaned answers and contexts
        cleaned_answers = []
        contexts = []

        # Iterate over each row in the DataFrame
        for index, row in df.iterrows():
            answer_text = row['answer']

            # Split the answer text based on the markers <|im_start|> and <|im_end|>
            if "<start_of_turn>user" in answer_text and "<start_of_turn>model" in answer_text:
                # Extract context and the cleaned answer
                context_part = answer_text.split("<end_of_turn><start_of_turn>context")[1].split("<end_of_turn>")[0].strip()
                answer_part = answer_text.split("<start_of_turn>model")[1].strip()

                # Append to the lists
                cleaned_answers.append(answer_part)
                contexts.append([context_part])
            else:
                # If markers are not found, append the original content
                cleaned_answers.append(answer_text)
                contexts.append([""])

        print(len(contexts))
        # Create a new dictionary with cleaned answers and contexts
        dictionary = {
            "question": df['question'].tolist(),
            "ground_truth": df['ground_truths'].tolist(),
            "answer": cleaned_answers,
            "contexts": contexts

        }
        try:
            #create csv file
            save_data = pd.DataFrame.from_dict(dictionary)
            save_path = f"{model_name}_data_cleaned.csv"
            save_data.to_csv(save_path, index=False)
        except Exception as e:
            logging.error("Error saving the data: {}".format(e))

    logging.info("Data cleaning completed successfully.")

proc_data_created_by_gemma()

25


In [ ]:
df_gemma_cleaned = pd.read_csv('/content/gemma_data_cleaned.csv')
df_gemma_cleaned.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,This text describes the winds in the Ligurian ...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channel of the port of Imper is 12 and...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,The Golfo Marconi is situated between Punta di...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the Port of Ischia is u...,['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_gemma_cleaned['answer'][0]

'This text describes the winds in the Ligurian Sea and Golfe di Genova, stating that the prevailing wind is from the NW.'

In [ ]:
df_gemma_cleaned['contexts'][0]

"['French border) to Genova, then SE and S to Promontorio diPiombino and Arcipelago Toscano. \\nGeneral Remarks\\n10.1 Winds—Weather.— In the Ligurian Sea and Golfe di\\nGenova, the prevailing wind is from the NW practically all\\nyear. However, locally along the coast, there is some variation\\ndue to the land configuration. In the N part of the Tyrrhenian\\n\\nPunta della Madonnetta, 1 mile WNW, and consists of two har-bors, Porto del Valle, and Porto Vecchio.\\n13.9Winds—Weather.— Winds from the NW are often strong\\nand raise a sea in the roadst ead. Winds from the NE quadrant,\\nespecially NNE winds, are dangerous to vessels at the quays.\\n13.9Depths—Limitations.— The entrance to Porto del Valle has']"

In [ ]:
df_gemma_cleaned.to_csv('gemma_data_cleaned.csv', index=False)

In [ ]:
df_gemma_cleaned = pd.read_csv('/content/gemma_data_cleaned.csv')
df_gemma_cleaned.head()

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,This text describes the winds in the Ligurian ...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channel of the port of Imper is 12 and...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,The Golfo Marconi is situated between Punta di...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the Port of Ischia is u...,['Porto d’Ischia—Contact Information\nPort\nVH...


In [ ]:
df_1_5['contexts'][0]

"['French border) to Genova, then SE and S to Promontorio diPiombino and Arcipelago Toscano. \\nGeneral Remarks\\n10.1 Winds—Weather.— In the Ligurian Sea and Golfe di\\nGenova, the prevailing wind is from the NW practically all\\nyear. However, locally along the coast, there is some variation\\ndue to the land configuration. In the N part of the Tyrrhenian\\n\\nPunta della Madonnetta, 1 mile WNW, and consists of two har-bors, Porto del Valle, and Porto Vecchio.\\n13.9Winds—Weather.— Winds from the NW are often strong\\nand raise a sea in the roadst ead. Winds from the NE quadrant,\\nespecially NNE winds, are dangerous to vessels at the quays.\\n13.9Depths—Limitations.— The entrance to Porto del Valle has']"

In [ ]:
df_0_5.to_csv('/gdrive/MyDrive/Tesi_magistrale/qwen_0.5b_data_cleaned.csv', index=False)
df_1_5.to_csv('/gdrive/MyDrive/Tesi_magistrale/qwen_1.5b_data_cleaned.csv', index=False)
df_cleaned.to_csv('/gdrive/MyDrive/Tesi_magistrale/qwen_3b_data_cleaned.csv', index=False)
df_phi_int4_cleaned.to_csv('/gdrive/MyDrive/Tesi_magistrale/phi_int4_data_cleaned.csv', index=False)
df_phi_int8_cleaned.to_csv('/gdrive/MyDrive/Tesi_magistrale/phi_int8_data_cleaned.csv', index=False)
df_gemma_cleaned.to_csv('/gdrive/MyDrive/Tesi_magistrale/gemma_data_cleaned.csv', index=False)

#RAGAS


In [ ]:
!pip install ragas llama-index langchain-together --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/13

In [ ]:
from datasets import load_dataset
from datasets import Dataset
import ast
from ragas import EvaluationDataset
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity, LLMContextPrecisionWithReference, LLMContextPrecisionWithoutReference, ResponseRelevancy, NonLLMContextRecall
from ragas import evaluate
from langchain_together import Together
from langchain_together.embeddings import TogetherEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_together import TogetherEmbeddings
from langchain_together import ChatTogether
from google.colab import userdata

In [ ]:
df_gemma_cleaned = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/gemma_data_cleaned.csv')
df_phi_int4_cleaned = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/phi_int4_data_cleaned.csv')
df_phi_int8_cleaned = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/phi_int8_data_cleaned.csv')
df_0_5 = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/qwen_0.5b_data_cleaned.csv')
df_1_5 = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/qwen_1.5b_data_cleaned.csv')
df_3= pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/qwen_3b_data_cleaned.csv')

In [ ]:
df_phi_int4_cleaned['response'][15]

'The marrobbio phenomenon refers to the sudden and intense coastal flooding caused by strong storm surges, typically associated with tropical cyclones or intense low-pressure systems. This can lead to significant damage to coastal infrastructure and pose risks to human life. The term "marrobbio" is derived from the Italian word "mare," meaning sea, indicating its marine origin.'

In [ ]:
df_0_5['answer'][9]

'Yes, it is possible to land on the island of Pianosa in the Tremiti Islands. The Tremiti Islands, located off the coast of Venice, are known for their picturesque beaches and offer various recreational activities, including fishing and sailing. However, landing on these islands requires permission from local authorities and may involve navigating through narrow waters and coastal areas.'

In [ ]:
df_1_5['answer'][9]

"Is there any chance to visit the island of Pianosa within the Tremiti Islands?\nContext:\nThe Tremiti Islands (also known as the Salento Islands) form an archipelago off the coast of Italy's southern Apulia region. They consist of several small islands including Pianosa, which lies approximately 30 km south-southeast of Licata, Sicily. While not directly accessible by road from mainland Italy due to its location, tourists can reach Pianosa via ferry from nearby Calafrio, Sicily. The island itself has been used for military purposes since World War II and is now protected as a nature reserve. Visitors typically spend their"

In [ ]:
df_3['answer'][9]

'The island of Pianosa is part of the Archipelago di Capo Passero and is not accessible for landing due to its strategic importance and security measures. It is not open to public visits or landings.'

In [ ]:
df_1_5

,question,ground_truth,answer,contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,Is it true that Genoa experiences strong winds...,"['French border) to Genova, then SE and S to P..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,What is the depth of the sea in the port of Al...,"[""No. 39070), a small port, lies at the SW end..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,What is the VHF channel for the port of Imperi...,['LAT 0.0m\nNote: Predicted heights are in me...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,Could you please provide more information on t...,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N...."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"Thank you for the information, but I'm actuall...",['Porto d’Ischia—Contact Information\nPort\nVH...
5,Can you illustrate how one should enter the po...,To enter the port of genoa one must have a mod...,Can you provide guidance on entering the port ...,['passage through Canale di San Pietro. Pilots...
6,What is the intesity of the tides in the port ...,Tides in the port of Ancona are generally negl...,What is the intensity of the tides in the port...,"[""18.18Caution.— Landslides may occur along th..."
7,What are the main dangers in the port of Naples ?,The main dangers in the Port Of Naples are the...,Can you provide some information on the histor...,['']
8,What winds are sheltered from in the harbor of...,To all except N winds that can be very violent...,What wind conditions can be expected at Otrant...,['15.15Winds—Weather.— The harbor is sheltered...
9,Is it possible to land on the island of Pianos...,No because it is a marine protected area. One ...,Is there any chance to visit the island of Pia...,['']


In [ ]:
#change the names of the columns
df_gemma_cleaned.rename(columns={'question': 'user_input', 'ground_truth': 'reference', 'answer': 'response', 'contexts': 'retrieved_contexts'}, inplace=True)
df_phi_int4_cleaned.rename(columns={'question': 'user_input', 'ground_truth': 'reference', 'answer': 'response', 'contexts': 'retrieved_contexts'}, inplace=True)
df_phi_int8_cleaned.rename(columns={'question': 'user_input', 'ground_truth': 'reference', 'answer': 'response', 'contexts': 'retrieved_contexts'}, inplace=True)
df_0_5.rename(columns={'question': 'user_input', 'ground_truth': 'reference', 'answer': 'response', 'contexts': 'retrieved_contexts'}, inplace=True)
df_1_5.rename(columns={'question': 'user_input', 'ground_truth': 'reference', 'answer': 'response', 'contexts': 'retrieved_contexts'}, inplace=True)
df_3.rename(columns={'question': 'user_input', 'ground_truth': 'reference', 'answer': 'response', 'contexts': 'retrieved_contexts'}, inplace=True)

In [ ]:
df_gemma_cleaned['retrieved_contexts'] = df_gemma_cleaned['retrieved_contexts'].apply(ast.literal_eval)
df_phi_int8_cleaned['retrieved_contexts'] = df_phi_int8_cleaned['retrieved_contexts'].apply(ast.literal_eval)
df_phi_int4_cleaned['retrieved_contexts'] = df_phi_int4_cleaned['retrieved_contexts'].apply(ast.literal_eval)
df_0_5['retrieved_contexts'] = df_0_5['retrieved_contexts'].apply(ast.literal_eval)
df_1_5['retrieved_contexts'] = df_1_5['retrieved_contexts'].apply(ast.literal_eval)
df_3['retrieved_contexts'] = df_3['retrieved_contexts'].apply(ast.literal_eval)
type(df_0_5['retrieved_contexts'][0])

list

df_0_5 ha un Nan che ho tolto portando il numero di righe a 24 tanto non si poteva avere tutte le metriche e quelle poche calcolabili erano sfalsate

#Together AI

In [ ]:
together_completion = ChatTogether(
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    temperature=0.7,
    max_tokens=50000, #50000, vanno bene per semantic similarity
    timeout=100,
    max_retries=2,
    api_key=userdata.get('TOGETHER_API_TOKEN')
)


In [ ]:
together_completion.max_tokens

50000

In [ ]:
embed = TogetherEmbeddings(
    model="togethercomputer/m2-bert-80M-8k-retrieval",
    api_key=userdata.get("TOGETHER_API_TOKEN"),
    request_timeout = 1,
    max_retries=2
)

In [ ]:
together_llm = LangchainLLMWrapper(together_completion)
together_embeddings = LangchainEmbeddingsWrapper(embed)

In [ ]:
metrics = [
    LLMContextPrecisionWithoutReference (llm=together_llm),
    LLMContextRecall(llm = together_llm),
    FactualCorrectness(llm=together_llm),
    Faithfulness(llm=together_llm),
    ResponseRelevancy(embeddings=together_embeddings),
    SemanticSimilarity(embeddings=together_embeddings)
]

In [ ]:
df_1_5.head()

,user_input,reference,response,retrieved_contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,Is it true that Genoa experiences strong winds...,"[French border) to Genova, then SE and S to Pr..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,What is the depth of the sea in the port of Al...,"[No. 39070), a small port, lies at the SW end ..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,What is the VHF channel for the port of Imperi...,[LAT 0.0m\nNote: Predicted heights are in met...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,Could you please provide more information on t...,"[Golfo Marconi\n10.20 Golfo Marconi (44°18'N.,..."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,"Thank you for the information, but I'm actuall...",[Porto d’Ischia—Contact Information\nPort\nVHF...


In [ ]:
df_1_5.isna().sum()


,0
user_input,0
reference,0
response,0
retrieved_contexts,0


In [ ]:
# df_0_5 = df_0_5.dropna()
# df_0_5.isna().sum()

In [ ]:
df_gemma_cleaned.head()

,user_input,reference,response,retrieved_contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,This text describes the winds in the Ligurian ...,"[French border) to Genova, then SE and S to Pr..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[No. 39070), a small port, lies at the SW end ..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channel of the port of Imper is 12 and...,[LAT 0.0m\nNote: Predicted heights are in met...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,The Golfo Marconi is situated between Punta di...,"[Golfo Marconi\n10.20 Golfo Marconi (44°18'N.,..."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the Port of Ischia is u...,[Porto d’Ischia—Contact Information\nPort\nVHF...


In [ ]:
df_gemma_cleaned.isna().sum()

,0
user_input,0
reference,0
response,0
retrieved_contexts,0


In [ ]:
df_phi_int4_cleaned.head()

,user_input,reference,response,retrieved_contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,The typical winds in Genoa are predominantly f...,"[French border) to Genova, then SE and S to Pr..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio ra...,"[No. 39070), a small port, lies at the SW end ..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channels of the port of Imperia are ch...,[LAT 0.0m\nNote: Predicted heights are in met...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,The Golfo Marconi is situated between Punta di...,"[Golfo Marconi\n10.20 Golfo Marconi (44°18'N.,..."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the port of Ischia is uc...,[Porto d’Ischia—Contact Information\nPort\nVHF...


In [ ]:
df_phi_int4_cleaned.isna().sum()

,0
user_input,0
reference,0
response,0
retrieved_contexts,0


In [ ]:
df_phi_int8_cleaned.head()

,user_input,reference,response,retrieved_contexts
0,What are the typical winds in Genoa ?,The typical winds in Genoa are in winter from ...,The typical winds in Genoa are predominantly f...,"[French border) to Genova, then SE and S to Pr..."
1,What is the depth of the sea in the port of Al...,The average depth in the port of Alassio is 4m,The depth of the sea in the port of Alassio is...,"[No. 39070), a small port, lies at the SW end ..."
2,What is the VHF channel of the port of imperia ?,There are two VHF channels in the port of Impe...,The VHF channels of the port of Imperia are ch...,[LAT 0.0m\nNote: Predicted heights are in met...
3,Where is situated the Golfo Marconi ?,Marconi Gulf is located between Portofino and ...,"The Golfo Marconi is situated at 44°18'N., 9°1...","[Golfo Marconi\n10.20 Golfo Marconi (44°18'N.,..."
4,Can you tell me the email address of the port ...,Ischia Harbor Email is ischia@guardiacostiera.it,The email address for the port of Ischia is uc...,[Porto d’Ischia—Contact Information\nPort\nVHF...


In [ ]:
df_phi_int8_cleaned.isna().sum()

,0
user_input,0
reference,0
response,0
retrieved_contexts,0


In [ ]:
dataset = Dataset.from_pandas(df_phi_int8_cleaned) #change the dataframe to compute the metrics
eval_dataset = EvaluationDataset.from_hf_dataset(dataset)
eval_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=25)

In [ ]:
#the dataset of qwen 0.5b have 24 rows instead of 25 because the 4 row have a nan as a response
import time
def compute_metrics_with_together(iterations, model_name):
  for i in range(1,iterations+1):
    print(i)
    results = evaluate(dataset=eval_dataset, metrics=metrics, llm=together_llm, embeddings=together_embeddings)
    print(f"results computed: {i}")
    df = results.to_pandas()
    df.to_csv(f'/gdrive/MyDrive/Tesi_magistrale/metrics/together/{model_name}_metrics_together_{i}.csv', index=False)
    print("results saved")

    # Only sleep if it's not the last iteration
    if i < iterations:
        time.sleep(60)
  return "done"

In [ ]:
compute_metrics_with_together(5, 'phi_int8')

1


Evaluating:   0%|          | 0/150 [00:00<?, ?it/s]

results computed: 1
results saved
2


Evaluating:   0%|          | 0/150 [00:00<?, ?it/s]

results computed: 2
results saved
3


...

Evaluating:   0%|          | 0/150 [00:00<?, ?it/s]

results computed: 4
results saved
5


Evaluating:   0%|          | 0/150 [00:00<?, ?it/s]

results computed: 5
results saved


'done'

In [ ]:
results_together_all_metrics

{'llm_context_precision_without_reference': 0.6400, 'context_recall': 0.2900, 'factual_correctness': 0.2760, 'faithfulness': 0.5484, 'answer_relevancy': 0.6699, 'semantic_similarity': 0.6552}

In [ ]:
df_together_all_metrics_qwen_3b = results_together_all_metrics.to_pandas()
df_together_all_metrics_qwen_3b

,user_input,retrieved_contexts,response,reference,llm_context_precision_without_reference,context_recall,factual_correctness,faithfulness,answer_relevancy,semantic_similarity
0,What are the typical winds in Genoa ?,"[French border) to Genova, then SE and S to Pr...","Based on the provided context, the prevailing ...",The typical winds in Genoa are in winter from ...,1.0,0.00,0.00,0.666667,0.573250,0.796118
1,What is the depth of the sea in the port of Al...,"[No. 39070), a small port, lies at the SW end ...",The depth of the sea in the port of Alassio is...,The average depth in the port of Alassio is 4m,1.0,0.00,0.00,0.500000,1.000000,0.868037
2,What is the VHF channel of the port of imperia ?,[LAT 0.0m\nNote: Predicted heights are in met...,"According to the provided information, the VHF...",There are two VHF channels in the port of Impe...,1.0,1.00,1.00,1.000000,0.933306,0.876434
3,Where is situated the Golfo Marconi ?,"[Golfo Marconi\n10.20 Golfo Marconi (44°18'N.,...",Where is situated the Golfo Marconi?\nThe Golf...,Marconi Gulf is located between Portofino and ...,1.0,1.00,0.57,0.800000,0.961230,0.559637
4,Can you tell me the email address of the port ...,[Porto d’Ischia—Contact Information\nPort\nVHF...,"Based on the provided contact information, the...",Ischia Harbor Email is ischia@guardiacostiera.it,1.0,1.00,0.00,1.000000,0.950797,0.168339
5,Can you illustrate how one should enter the po...,[passage through Canale di San Pietro. Pilots ...,"To enter the port of Genoa, vessels should app...",To enter the port of genoa one must have a mod...,1.0,0.50,0.00,0.333333,0.771166,0.892141
6,What is the intesity of the tides in the port ...,[18.18Caution.— Landslides may occur along the...,"I apologize, but the provided context does not...",Tides in the port of Ancona are generally negl...,0.0,0.00,0.00,0.600000,0.000000,0.873553
7,What are the main dangers in the port of Naples ?,[],The main dangers in the port of Naples include...,The main dangers in the Port Of Naples are the...,0.0,0.00,0.50,0.000000,1.000000,0.832222
8,What winds are sheltered from in the harbor of...,[15.15Winds—Weather.— The harbor is sheltered ...,"Based on the provided context, the harbor of O...",To all except N winds that can be very violent...,0.0,0.00,0.25,1.000000,0.875728,0.476168
9,Is it possible to land on the island of Pianos...,[],The island of Pianosa is part of the Archipela...,No because it is a marine protected area. One ...,0.0,0.00,0.29,0.000000,0.784378,0.628549


In [ ]:
df_together_all_metrics_qwen_3b.to_csv('/gdrive/MyDrive/Tesi_magistrale/metrics/qwen_3b_metrics_together_1.csv', index=False)

LLMContextPrecisionWithReference se metto il max_tokens a 100000  mi dice che è troppo
 (ERROR:ragas.executor:Exception raised in Job[1]: UnprocessableEntityError(Error code: 422 - {'error': {'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 1432 `inputs` tokens and 100000 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}})
)

 e se metto 30000 è troppo

 ERROR:ragas.executor:Exception raised in Job[2]: LLMDidNotFinishException(The LLM generation was not completed. Please increase try increasing the max_tokens and try again.)


#OPENAI

In [ ]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o", api_key=userdata.get('OPENAI_API_KEY')))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model = 'text-embedding-3-small', openai_api_key = userdata.get('OPENAI_API_KEY')))

In [ ]:
evaluator_embeddings.embeddings

In [ ]:
metrics = [
    LLMContextPrecisionWithoutReference (llm=evaluator_llm),
    LLMContextRecall(llm = evaluator_llm),
    FactualCorrectness(llm=evaluator_llm),
    Faithfulness(llm=evaluator_llm),
    ResponseRelevancy(embeddings=evaluator_embeddings),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]

In [ ]:
df_0_5.isna().sum()

In [ ]:
df_0_5 = df_0_5.dropna()

In [ ]:
df_0_5.isna().sum()

In [ ]:
df_0_5.head()

In [ ]:
df_1_5.head()

In [ ]:
df_1_5.isna().sum()

In [ ]:
df_gemma_cleaned.head()

In [ ]:
df_gemma_cleaned.isna().sum()

In [ ]:
df_phi_int4_cleaned.head()

In [ ]:
df_phi_int4_cleaned.isna().sum()

In [ ]:
df_phi_int8_cleaned.head()

In [ ]:
df_phi_int8_cleaned.isna().sum()

In [ ]:
dataset = Dataset.from_pandas(df_phi_int8_cleaned) #change the dataframe to compute the metrics
eval_dataset = EvaluationDataset.from_hf_dataset(dataset)
eval_dataset

In [ ]:
results = evaluate(dataset=eval_dataset, metrics=metrics, llm=evaluator_llm, embeddings=evaluator_embeddings)

In [ ]:
results

In [ ]:
df = results.to_pandas()
df.head()

In [ ]:
df.to_csv('/gdrive/MyDrive/Tesi_magistrale/metrics/OpenAI/phi_int8_metrics.csv', index=False)

OpenAI: {'llm_context_precision_without_reference': 0.6000, 'context_recall': 0.2500, 'factual_correctness': 0.1900, 'faithfulness': 0.5734, 'answer_relevancy': 0.6905, 'semantic_similarity': 0.6488}

together (llama 3.1 70b turbo instruct FP8): {'llm_context_precision_without_reference': 0.6400, 'context_recall': 0.2900, 'factual_correctness': 0.2760, 'faithfulness': 0.5484, 'answer_relevancy': 0.6699, 'semantic_similarity': 0.6552}





#COMPUTE THE MEAN

In [ ]:
import glob
import os
import pandas as pd

def compute_metrics_means(folder_path, starting_string, metrics_columns):
    """
    Computes the mean of specified metrics across multiple CSV files in a folder.

    Parameters:
    - folder_path (str): Path to the folder containing the CSV files.
    - starting_string (str): Starting string to filter the CSV files.
    - metrics_columns (list): List of metric column names to compute means for.

    Returns:
    - pd.DataFrame: A DataFrame with the mean values of each metric.
    """
    # Initialize a dictionary to store lists for each metric's values
    metrics_values = {metric: [] for metric in metrics_columns}

    # Get all files that start with the specified string
    files = glob.glob(os.path.join(folder_path, starting_string + '*'))

    # Process each file
    for file in files:
        print(f"Processing file: {file}")
        # Load the CSV file as a dataframe
        df = pd.read_csv(file)
        # Collect values for each metric if the column exists in the dataframe
        for metric in metrics_columns:
            if metric in df.columns:
                metrics_values[metric].extend(df[metric])
            else:
                print(f"Warning: '{metric}' column not found in {file}")

    # Calculate the mean for each metric and save in a new DataFrame
    metrics_mean = {metric: pd.Series(values).mean() for metric, values in metrics_values.items()}
    mean_df = pd.DataFrame([metrics_mean])
    mean_df.to_csv(f'{folder_path}/mean_metric/{starting_string}_mean_metrics.csv')
    print("results saved")

    return mean_df



In [ ]:
# Example usage
folder_path = '/gdrive/MyDrive/Tesi_magistrale/metrics/together'
starting_string = 'phi_int8' # change the name model to compute different mean for different dataframes
metrics_columns = [
    'llm_context_precision_without_reference',
    'context_recall',
    'factual_correctness',
    'faithfulness',
    'answer_relevancy',
    'semantic_similarity'
]

# Call the function and get the means DataFrame
mean_df = compute_metrics_means(folder_path, starting_string, metrics_columns)
mean_df

Processing file: /gdrive/MyDrive/Tesi_magistrale/metrics/together/phi_int8_metrics_together_1.csv
Processing file: /gdrive/MyDrive/Tesi_magistrale/metrics/together/phi_int8_metrics_together_2.csv
Processing file: /gdrive/MyDrive/Tesi_magistrale/metrics/together/phi_int8_metrics_together_3.csv
Processing file: /gdrive/MyDrive/Tesi_magistrale/metrics/together/phi_int8_metrics_together_4.csv
Processing file: /gdrive/MyDrive/Tesi_magistrale/metrics/together/phi_int8_metrics_together_5.csv
results saved


,llm_context_precision_without_reference,context_recall,factual_correctness,faithfulness,answer_relevancy,semantic_similarity
0,0.72,0.29,0.26128,0.418838,0.777359,0.670621


#Prove

In [ ]:
df_test_1 = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/metrics/qwen_3b_metrics.csv')
df_test_1.head()

,user_input,retrieved_contexts,response,reference,llm_context_precision_without_reference,context_recall,factual_correctness,faithfulness,answer_relevancy,semantic_similarity
0,What are the typical winds in Genoa ?,"['French border) to Genova, then SE and S to P...","Based on the provided context, the prevailing ...",The typical winds in Genoa are in winter from ...,1.0,0.0,0.00,0.666667,0.730720,0.751041
1,What is the depth of the sea in the port of Al...,"[""No. 39070), a small port, lies at the SW end...",The depth of the sea in the port of Alassio is...,The average depth in the port of Alassio is 4m,1.0,0.0,0.00,0.500000,0.995559,0.831495
2,What is the VHF channel of the port of imperia ?,['LAT 0.0m\nNote: Predicted heights are in me...,"According to the provided information, the VHF...",There are two VHF channels in the port of Impe...,1.0,1.0,1.00,1.000000,0.915042,0.910345
3,Where is situated the Golfo Marconi ?,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N....",Where is situated the Golfo Marconi?\nThe Golf...,Marconi Gulf is located between Portofino and ...,1.0,0.0,0.57,1.000000,0.957414,0.725466
4,Can you tell me the email address of the port ...,['Porto d’Ischia—Contact Information\nPort\nVH...,"Based on the provided contact information, the...",Ischia Harbor Email is ischia@guardiacostiera.it,1.0,1.0,0.00,1.000000,0.959090,0.750086


In [ ]:
df_test_2_together = pd.read_csv('/gdrive/MyDrive/Tesi_magistrale/metrics/qwen_3b_metrics_together_1.csv')
df_test_2_together.head()

,user_input,retrieved_contexts,response,reference,llm_context_precision_without_reference,context_recall,factual_correctness,faithfulness,answer_relevancy,semantic_similarity
0,What are the typical winds in Genoa ?,"['French border) to Genova, then SE and S to P...","Based on the provided context, the prevailing ...",The typical winds in Genoa are in winter from ...,1.0,0.0,0.00,0.666667,0.573250,0.796118
1,What is the depth of the sea in the port of Al...,"[""No. 39070), a small port, lies at the SW end...",The depth of the sea in the port of Alassio is...,The average depth in the port of Alassio is 4m,1.0,0.0,0.00,0.500000,1.000000,0.868037
2,What is the VHF channel of the port of imperia ?,['LAT 0.0m\nNote: Predicted heights are in me...,"According to the provided information, the VHF...",There are two VHF channels in the port of Impe...,1.0,1.0,1.00,1.000000,0.933306,0.876434
3,Where is situated the Golfo Marconi ?,"[""Golfo Marconi\n10.20 Golfo Marconi (44°18'N....",Where is situated the Golfo Marconi?\nThe Golf...,Marconi Gulf is located between Portofino and ...,1.0,1.0,0.57,0.800000,0.961230,0.559637
4,Can you tell me the email address of the port ...,['Porto d’Ischia—Contact Information\nPort\nVH...,"Based on the provided contact information, the...",Ischia Harbor Email is ischia@guardiacostiera.it,1.0,1.0,0.00,1.000000,0.950797,0.168339


In [ ]:
faithfulness_values = pd.concat([df['faithfulness'] for df in [df_test_1, df_test_2_together]])
faithfulness_mean = faithfulness_values.mean()
print(f"The mean of 'faithfulness' across the five dataframes is: {faithfulness_mean}")


The mean of 'faithfulness' across the five dataframes is: 0.5608888888888889
